<a href="https://colab.research.google.com/github/rineeee/Stock_Price_Check/blob/main/stock_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import matplotlib.pyplot as plt
from flask import Flask
from bs4 import BeautifulSoup
import urllib.request as req
import FinanceDataReader as fdr
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)


code = '005930'
stock = fdr.DataReader(code)

scaler = MinMaxScaler().fit(stock)
scaled = scaler.transform(stock)
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Change']
df = pd.DataFrame(scaled, columns=scale_cols)

feature_cols = ['Open', 'High', 'Low', 'Volume']
label_cols = ['Close']

train_feature = df[feature_cols]  #  N by 4
train_label = df[label_cols]      # N by 1


train_feature, train_label = make_dataset(train_feature, train_label, 20)

print(train_feature.shape)
print(train_label.shape)

print(train_feature[0, :, :])
print(train_label[0])


model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

history = model.fit(train_feature, train_label,  epochs=2,  batch_size=16) 

p = model.predict(train_feature[-1,:,:].reshape(1,20,4))
p = scaler.inverse_transform([[0,0,0,p[0][0],0,0]])[0][3]
print(p)

(5980, 20, 4)
(5980, 1)
[[0.01081949 0.01040289 0.01082682 0.0079453 ]
 [0.01091916 0.01018595 0.01043575 0.00373086]
 [0.010299   0.00974174 0.00993296 0.00656091]
 [0.00984496 0.00918388 0.00935196 0.00460569]
 [0.00947951 0.00924587 0.00946369 0.00327236]
 [0.00978959 0.00955579 0.0098324  0.0052833 ]
 [0.01050941 0.01       0.01004469 0.00432944]
 [0.00988926 0.00922521 0.0093743  0.00430557]
 [0.00968992 0.00909091 0.00883799 0.01091672]
 [0.00883721 0.00835744 0.00858101 0.01216044]
 [0.00834994 0.00797521 0.00821229 0.00725554]
 [0.00797342 0.00795455 0.00804469 0.00422983]
 [0.00807309 0.00784091 0.00789944 0.00318248]
 [0.00782946 0.00766529 0.00758659 0.0053766 ]
 [0.00820598 0.00820248 0.0081676  0.00972994]
 [0.0092691  0.00918388 0.0092514  0.01137025]
 [0.00983389 0.00922521 0.00906145 0.01151271]
 [0.00951274 0.0090186  0.00915084 0.0031814 ]
 [0.00968992 0.00903926 0.0092514  0.00467111]
 [0.00880399 0.00834711 0.00852514 0.00525241]]
[0.00143866]
Epoch 1/2
374/374 [===

60900.151998996735


In [ ]:
data = {}
data["20220806"] = p
with open 